In [2]:
from datetime import datetime
from dotenv import load_dotenv
import numpy as np
import os
import pandas as pd
from pathlib import Path
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout


In [3]:
def timer(start_time=None):
    if start_time is not None:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

    else:
        start_time = datetime.now()
        return start_time

In [4]:
load_dotenv()
path = Path(os.getenv('PATH_DATASET2')).joinpath('publictracks_csv')

In [8]:
track_names = np.genfromtxt(path.joinpath('track_names.csv'), delimiter=',', encoding='utf-8')

In [9]:
tot_num = len(track_names)
print('Total number fo tracks: ', tot_num)

Total number fo tracks:  17760


In [37]:
f_paths = []
for f in path.iterdir():
    f_paths.append(f)  

In [38]:
f_paths.remove(f_paths[3722])

In [39]:
# read_csv files and create a list of data arrays
data= []
tr_name = []
for f in f_paths:
    data.append(np.genfromtxt(f, delimiter=','))
    tr_name.append(f.name)

In [44]:
print('Number of tracks:',len(data))
print('Length of sample input array:', len(data[0]))
if len(tr_name) == len(data) : print('Number of track name and track arrays match')  
else: print("Number of tracks don't match")
print('Sample track name:', tr_name[1])

Number of tracks: 17651
Length of sample input array: 1200
Number of tracks match
Sample track name: &Run-Sir.csv


In [49]:
x_train, x_test, tr_name_train, tr_name_test = train_test_split(data, tr_name, test_size = 0.25, random_state=17)

In [114]:
# cast lists to numpy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)

In [115]:
x_val = x_train[-2000:]
tr_name_val = tr_name_train[-2000:]
x_train = x_train[:-2000]
tr_name_train = tr_name_train[:-2000]

In [116]:
print('Number of tracks in training:', len(x_train))
print('Number of tracks in validation:',len(x_val))
print('Number of tracks in testing:',len(x_test))

Number of tracks in training: 9238
Number of tracks in validation: 2000
Number of tracks in testing: 4413


In [117]:
def autoencoder_functional(inputlength, latent_dim, layer_nodes, activations=None, dropout=False):

    def encdec_layer(x, nodes, activation='relu', bn=True, dropout=dropout):
    
        # define a encoding/decoding layer
        
        x = Dense(nodes)(x)
        x = Activation(activation)(x)
        if bn:
            x = BatchNormalization()(x)
        if dropout:
            x = Dropout(0.25)(x)
        
        return x    
    
    # DEFAULT ACTIVATIONS
    acts = ['relu'] * len(layer_nodes)
    
    if activations:
        if len(activations) == len(layer_nodes):
            acts = activations
        else:
            print("Number of activations must be equal to number of layers")
    
    # ASSEMBLING ENCODER MODEL
    enc_inputs = Input(shape=(inputlength,))
    x = encdec_layer(enc_inputs, layer_nodes[0], activation=acts[0])
    
    # add encoding layers for remainder of depth
    for layer_ in range(1, len(layer_nodes)):
        x = encdec_layer(x, layer_nodes[layer_], activation=acts[layer_])
    
    # Last encoding layer with number of nodes = dimension of latent feature space
    enc_latent = encdec_layer(x, latent_dim, activation='tanh', dropout=True)
    
    # ASSEMBLING DECODER MODEL
    # Decoder model will be a mirror of the encoder model
    # Hence, we will be moving backwards through layer_nodes
    
    latent_inputs = Input(shape=(latent_dim,))
    x = encdec_layer(latent_inputs, layer_nodes[-1], activation=acts[-1])
    
    # add decoding layers for remainder of depth
    for layer_ in reversed(range(1, len(layer_nodes))):
        x = encdec_layer(x, layer_nodes[layer_], activation=acts[layer_])
    
    # Last decoding layer with number of nodes = original input dimension
    dec_outputs = encdec_layer(x, inputlength, activation='tanh', dropout=True)
    
    # Create encoder and decoder model
    encoder = Model(inputs=enc_inputs, outputs=enc_latent, name='encoder')
    decoder = Model(inputs=latent_inputs, outputs=dec_outputs, name = 'decoder')
    # Create autoencoder model
    autoencoder = Model(inputs=enc_inputs, outputs=decoder(encoder(enc_inputs)), name = 'autoencoder')
    
    return (autoencoder, encoder, decoder)

In [118]:
inputlength = len(x_train[0])
latent_dim = 50
layer_nodes = [400, 200, 100]
activations = ['relu', 'relu', 'tanh']

In [119]:
print('input length:', inputlength)
print('\nNumber of layers in encoder/decoder:', len(layer_nodes)+1)
print('Number of layers in autoencoder:', 2*(len(layer_nodes) + 1))
print('\nNOTE: Each encoding/decoding layer consists of a dense layer, activation layer \
\n      and may or may not have a batchnorm layer or a dropout layer.\n')
      
for i in range(len(layer_nodes)):
    print('Layer {} has {} nodes with activation - {}.'.format(i, layer_nodes[i], activations[i]))
print('Layer {} has  {} nodes with activation - tanh.'.format(len(layer_nodes), latent_dim))
print('\nThe last layer in the encoder gives us a reduced dimensional feature space.\n')

input length: 1200

Number of layers in encoder/decoder: 4
Number of layers in autoencoder: 8

NOTE: Each encoding/decoding layer consists of a dense layer, activation layer 
      and may or may not have a batchnorm layer or a dropout layer.

Layer 0 has 400 nodes with activation - relu.
Layer 1 has 200 nodes with activation - relu.
Layer 2 has 100 nodes with activation - tanh.
Layer 3 has  50 nodes with activation - tanh.

The last layer in the encoder gives us a reduced dimensional feature space.



In [210]:
tf.keras.backend.clear_session()
autoencoder, encoder, decoder = autoencoder_functional(inputlength, latent_dim =latent_dim, \
                                            layer_nodes=layer_nodes, activations=activations, dropout=True )

In [211]:
autoencoder.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [232]:
#encoder.summary()

In [233]:
#autoencoder.summary()

In [214]:
encoder.get_layer('dense_3').get_weights()

[array([[ 0.07347737,  0.0329092 ,  0.05429836, ...,  0.17816927,
          0.00850554,  0.01360936],
        [ 0.09909363,  0.06306238, -0.14623761, ...,  0.09917618,
         -0.11254063, -0.1408535 ],
        [-0.00163308, -0.17235552,  0.11359926, ..., -0.19663297,
         -0.19500351,  0.13404955],
        ...,
        [ 0.19081388,  0.14369269,  0.04819909, ...,  0.03154078,
         -0.11043105, -0.12191267],
        [ 0.10830842,  0.03466821, -0.05534701, ...,  0.05124713,
          0.00607271,  0.00878949],
        [-0.18584453, -0.05887942,  0.10120259, ...,  0.02133155,
         -0.06625824, -0.18255568]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)]

In [215]:
autoencoder.get_layer('encoder').get_layer('dense_3').get_weights()

[array([[ 0.07347737,  0.0329092 ,  0.05429836, ...,  0.17816927,
          0.00850554,  0.01360936],
        [ 0.09909363,  0.06306238, -0.14623761, ...,  0.09917618,
         -0.11254063, -0.1408535 ],
        [-0.00163308, -0.17235552,  0.11359926, ..., -0.19663297,
         -0.19500351,  0.13404955],
        ...,
        [ 0.19081388,  0.14369269,  0.04819909, ...,  0.03154078,
         -0.11043105, -0.12191267],
        [ 0.10830842,  0.03466821, -0.05534701, ...,  0.05124713,
          0.00607271,  0.00878949],
        [-0.18584453, -0.05887942,  0.10120259, ...,  0.02133155,
         -0.06625824, -0.18255568]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)]

In [216]:
BATCH_SIZE = 64
TRAIN_STEPS_PER_EPOCH = np.ceil(len(x_train)/ BATCH_SIZE)       #np.ceil(TRAIN_COUNT/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.ceil(len(x_val)/BATCH_SIZE)        #np.ceil(VAL_COUNT/BATCH_SIZE)
TEST_STEPS = len(x_test)
EPOCHS = 5

In [217]:
st_time = timer(None)
autoenc = autoencoder.fit(x=x_train, y=x_train,
                          steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
                          epochs = EPOCHS, shuffle=False,
                          validation_data = (x_val, x_val),
                          validation_steps= VAL_STEPS_PER_EPOCH)
timer(st_time)

Epoch 1/5
145/145 [==============================] - 254s 2s/step - loss: 1.1593 - mean_squared_error: 1.1593 - val_loss: 0.4720 - val_mean_squared_error: 0.4720
Epoch 2/5
145/145 [==============================] - 262s 2s/step - loss: 0.8110 - mean_squared_error: 0.8110 - val_loss: 0.3013 - val_mean_squared_error: 0.3013
Epoch 3/5
145/145 [==============================] - 270s 2s/step - loss: 0.4366 - mean_squared_error: 0.4366 - val_loss: 0.4719 - val_mean_squared_error: 0.4719
Epoch 4/5
145/145 [==============================] - 275s 2s/step - loss: 0.0745 - mean_squared_error: 0.0745 - val_loss: 0.1737 - val_mean_squared_error: 0.1737
Epoch 5/5
145/145 [==============================] - 274s 2s/step - loss: 0.0707 - mean_squared_error: 0.0707 - val_loss: 0.0862 - val_mean_squared_error: 0.0862

 Time taken: 0 hours 22 minutes and 16.9 seconds.


In [219]:
autoencoder.get_layer('encoder').get_layer('dense_3').get_weights()

[array([[ 0.07741034,  0.01946484,  0.02221563, ...,  0.11088157,
          0.06397545,  0.0207343 ],
        [ 0.05684531,  0.04776891, -0.11114325, ...,  0.08102795,
         -0.0382463 , -0.06827035],
        [-0.01745828, -0.08250307,  0.156531  , ..., -0.09705876,
         -0.1688261 ,  0.08050036],
        ...,
        [ 0.09508882,  0.11078127,  0.13488206, ...,  0.00396039,
         -0.16677007, -0.10692524],
        [ 0.06418883,  0.09210512, -0.0302702 , ...,  0.00326248,
          0.00720771,  0.02971781],
        [-0.12228182, -0.06985266,  0.0567733 , ...,  0.03428658,
         -0.04222217, -0.15125948]], dtype=float32),
 array([ 2.23267847e-03, -3.21475975e-02,  3.40749994e-02, -3.61994021e-02,
         1.69316394e-04,  1.67661294e-01, -5.06708864e-03, -1.29663473e-04,
         2.83424999e-03, -8.54090899e-02, -2.02340297e-02,  2.80528180e-02,
        -7.25839380e-03, -2.03876430e-03, -7.06078298e-03, -9.26623344e-02,
        -8.32440928e-02, -8.09156373e-02, -6.58074319e

In [224]:
encoder.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [228]:
encoded_out = encoder.predict(np.array([x_test[0]]))
encoded_out

array([[ 1.2110815 , -1.0433093 , -1.2517    ,  0.47041315, -0.7390007 ,
         0.6040875 , -1.1193643 ,  0.7933613 , -0.45897102, -0.9392003 ,
         1.1238505 ,  1.0214757 , -0.90243196, -1.0413766 , -1.1636659 ,
        -1.1826581 , -1.1321986 , -1.1559083 , -0.24298921, -1.2482964 ,
         0.21138465,  0.26654953, -1.1740204 , -0.6471349 ,  0.81743467,
        -1.2306262 , -0.41867378, -1.1011943 , -0.61266196, -0.9025874 ,
         0.58824086,  1.0680349 , -0.72454953,  0.528056  ,  0.88737035,
         0.5664663 , -0.41335368,  0.18181613,  0.34764242,  0.9378705 ,
        -1.0176069 ,  0.45071644,  1.3431733 ,  1.200258  ,  1.1496104 ,
        -0.9614325 , -0.02830141,  0.80525696,  0.8734684 , -0.94281787]],
      dtype=float32)

In [229]:
autoencoder.save(filepath=Path(os.getenv('PATH_DATASET2')).joinpath('autoencoder'))
encoder.save(filepath=Path(os.getenv('PATH_DATASET2')).joinpath('encoder'))

In [230]:
loaded_model = keras.models.load_model(filepath=Path(os.getenv('PATH_MODELS')).joinpath('encoder'))

In [165]:
#import graphviz
#import pydot
#keras.utils.plot_model(loaded_model)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
